# In the notebook, a model is trained for each medicine

data source: ./simplified_data/simplified_data2.csv

the last few blocks were commmented since I have no clue what they were for.




# 0. Global Variables

In [1]:
# Remove the entire column related to a specific medicine if its occurrence is below the defined threshold.
# In the given dataset, setting this threshold to 250 would retain columns for only the top 10 most frequently used medicines.
DeleteMedThreshold = 250

# Determine the number of medicines to be trained as output
# The total number of medicine is 102
NumMedTrain = 102

# Decide whether to enhance accuracy by utilizing class weights
UseClassWeight = True

# Decide learning_rate
LearningRate = 0.005

# 1. Import module


In [2]:

# Importing necessary libraries
import numpy as np
import pandas as pd
import statistics
from tabulate import tabulate
import matplotlib.pyplot as plt

# Importing TensorFlow for deep learning
import tensorflow
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from keras.callbacks import EarlyStopping, LambdaCallback
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy

# Importing scikit-learn for data preprocessing and utilities
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils.class_weight import compute_sample_weight

# Importing custom utility functions
from utility_file import my_utilities as myutil
from utility_file import load_data

# 2. Read Data
Load the data using the custom module "load_data"

In [3]:
# Load data for training a model with one specific medicine, including debugging information
(X_np, X_val_np, train_y, val_y, 
 num_col_x, num_1_valy, num_0_valy) = load_data.load_data_for_1_med_with_debug(del_med_thres=DeleteMedThreshold, random_seed=1, n=NumMedTrain)

# Ensure the correct data types for loaded variables
assert isinstance(X_np, np.ndarray)
assert isinstance(X_val_np, np.ndarray)
assert isinstance(train_y, pd.DataFrame)
assert isinstance(val_y, pd.DataFrame)
assert isinstance(num_col_x, int)


--------------------------------------------------------------------------------
ReadData:
Type of data: <class 'pandas.core.frame.DataFrame'>
Shape of data = (797 rows, 215 cols).
End of ReadData
--------------------------------------------------------------------------------
SplitXY:
Shape of X = (796 rows, 111 cols).
Shape of y = (796 rows, 102 cols).
End of SplitXY
--------------------------------------------------------------------------------
In load_data_for_1_med_with_debug of load_data.py, random_seed= 1
After SplitXY, total number of 0, 1 in y:
Number of 0s: 72318
Number of 1s: 8874
--------------------------------------------------------------------------------
DeleteMedicine: shape of y is (796, 10).
Train_X.shape:  (637, 111)
Train_y.shape:  (637, 10)

Split Training Validation
Number of 0s in train_y: 3660
Number of 1s train_y: 2710
Number of 0s in val_y: 860
Number of 1s val_y: 730
--------------------------------------------------------------------------------


# 3. Data Type Checking

In [4]:
# Uncomment the line below to display the DataFrame content and structure
# myutil.print_df(val_y, "---- y ----")

# Uncomment the line below to print the DataFrame directly
# print(val_y)

# Checking:

# Counting NA values in y
na_count = val_y.isna().sum().sum()

# Counting str values in y
str_count = val_y[val_y.map(type) == str].count().sum()

# Counting int values in y
int_count = val_y[val_y.map(type) == int].count().sum()

# Counting float values in y
float_count = val_y[val_y.map(type) == float].count().sum()

# Display the results
print(f"Number of NA values in y: {na_count}")
print(f"Number of str values in y: {str_count}")
print(f"Number of int values in y: {int_count}")
print(f"Number of float values in y: {float_count}")


Number of NA values in y: 0
Number of str values in y: 0
Number of int values in y: 1590
Number of float values in y: 0


# 4. Compute Class Weight

In [5]:
# Convert the 'train_y' DataFrame to a NumPy array
train_y_np = np.array(train_y)

# Determine the number of labels (columns) in the array
num_labels = train_y_np.shape[1]

# Initialize an empty dictionary to store class weights for each label
class_weight_dic = {}

# Iterate over each label column
for i in range(num_labels):
    # Count the occurrences of each class (0 and 1) in the current label column
    unique_values, counts = np.unique(train_y_np[:, i], return_counts=True)
    
    # Create a dictionary mapping class values to their frequencies
    value_frequency_dict = dict(zip(unique_values, counts))
    
    # Calculate the total number of occurrences for normalization
    total = value_frequency_dict.get(0, 0) + value_frequency_dict.get(1, 0)
    
    # Calculate class weights and store them in the dictionary
    class_weight_dic[i] = {0: (value_frequency_dict.get(1, 0) / total), 1: (value_frequency_dict.get(0, 0) / total)}

# Display the computed class weights
print(class_weight_dic)

{0: {0: 0.42543171114599687, 1: 0.5745682888540031}, 1: {0: 0.3218210361067504, 1: 0.6781789638932496}, 2: {0: 0.39717425431711145, 1: 0.6028257456828885}, 3: {0: 0.32653061224489793, 1: 0.673469387755102}, 4: {0: 0.35478806907378335, 1: 0.6452119309262166}, 5: {0: 0.4207221350078493, 1: 0.5792778649921507}, 6: {0: 0.5667189952904239, 1: 0.43328100470957615}, 7: {0: 0.33124018838304553, 1: 0.6687598116169545}, 8: {0: 0.5117739403453689, 1: 0.48822605965463106}, 9: {0: 0.598116169544741, 1: 0.40188383045525905}}


# 5. Build Model

In [6]:
# Define a Sequential model
model = Sequential([
    Dense(units=32, input_shape=(num_col_x,), activation='sigmoid'),
    # Additional layers (commented out for simplicity)
    # Dense(units=64, activation='relu'), 
    # Dense(units=128, activation='relu'), 
    # Dense(units=32, activation='relu'),
    # Dense(units=16, activation='sigmoid'), 
    Dense(units=2, activation='softmax')
])

# Display a summary of the model architecture
model.summary()

# Compile the model with specified optimizer, loss function, and metrics
model.compile(optimizer=Adam(learning_rate=LearningRate),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                3584      
                                                                 
 dense_1 (Dense)             (None, 2)                 66        
                                                                 
Total params: 3,650
Trainable params: 3,650
Non-trainable params: 0
_________________________________________________________________


# 6. Train Model
There will be a model for each medicine

In [7]:

# Initialize dictionaries to store results and training history
result_df_dict = {}        # Dictionary of DataFrames
accuracy_dict = {}         # Dictionary of accuracy for each medicine
prediction_train_dict = {}  # Dictionary of raw predictions for the training set
prediction_val_dict = {}    # Dictionary of raw predictions for the validation set

# Iterate over each medicine
for i in range(train_y.shape[1]):
    chosen_col = train_y.iloc[:, i].copy()
    
    # Ensure that the chosen column is a pandas Series
    assert(isinstance(chosen_col, pd.Series))
    assert(len(chosen_col) == len(train_y))
    
    print(f"Processing medicine {i + 1} of {train_y.shape[1]}: {chosen_col.name}")

    # Convert the chosen column to NumPy array
    chosen_y_np = chosen_col.values.astype('float64')

    # Copy the corresponding validation set column
    y_val_chosen_col = val_y.iloc[:, i].copy()
    
    # Ensure that the validation set column is a pandas Series
    assert(isinstance(y_val_chosen_col, pd.Series))
    assert(len(y_val_chosen_col) == len(val_y))

    # Early stopping callback
    early_stopping = EarlyStopping(monitor='loss', patience=30, restore_best_weights=True)

    # Fit the model for the current medicine
    Model = model.fit(
        x=X_np,
        y=chosen_y_np,
        class_weight=class_weight_dic[i] if UseClassWeight else None,
        epochs=5000,
        shuffle=True,
        verbose=0,
        callbacks=[early_stopping]
    )
    
    # Print when training stopped
    print(f"Training stopped at epoch {Model.epoch[-1]}")
    
    # Predict against the training set for diagnosing overfitting or underfitting
    predictions_train_set = model.predict(X_np)
    
    # Save raw result numpy array of training set to the dictionary
    prediction_train_dict[chosen_col.name] = predictions_train_set
    
    # Make predictions for the validation set
    predictions_val_set = model.predict(X_val_np)
    
    # Save raw result numpy array of validation set to the dictionary
    prediction_val_dict[chosen_col.name] = predictions_val_set
    
    # Plotting loss vs. epoch
    # plt.plot(Model.history['loss'], label='Training Loss')
    # plt.title('Loss vs. Epoch')
    # plt.xlabel('Epoch')
    # plt.ylabel('Loss')
    # plt.legend()
    # plt.show()

print("Training done.")


Processing medicine 1 of 10: 桂枝
Training stopped at epoch 478
5/5 [==============================] - 0s 500us/step
Processing medicine 2 of 10: 柴胡
Training stopped at epoch 405
5/5 [==============================] - 0s 500us/step
Processing medicine 3 of 10: 黃芩
Training stopped at epoch 293
5/5 [==============================] - 0s 750us/step
Processing medicine 4 of 10: 茯苓
Training stopped at epoch 287
5/5 [==============================] - 0s 750us/step
Processing medicine 5 of 10: 澤瀉
Training stopped at epoch 209
5/5 [==============================] - 0s 747us/step
Processing medicine 6 of 10: 附子
Training stopped at epoch 353
5/5 [==============================] - 0s 500us/step
Processing medicine 7 of 10: 甘草
Training stopped at epoch 348
5/5 [==============================] - 0s 750us/step
Processing medicine 8 of 10: 當歸
Training stopped at epoch 251
5/5 [==============================] - 0s 626us/step
Processing medicine 9 of 10: 白芍
Training stopped at epoch 281
5/5 [=============

# 7. Handle result

### 7.1 Calculate the f1 score of the training dataset and store the values in TrainMedicineDictioanry

In [8]:
# Calculate True Positives (TP), False Positives (FP), True Negatives (TN), and False Negatives (FN) for the training set
total_tp_train = 0
total_fp_train = 0
total_tn_train = 0
total_fn_train = 0

# Create a dictioanry to store all values of a medicine
TrainMedicineDictioanry = {}

# Iterate through each medicine's raw prediction array
for key, arr in prediction_train_dict.items():
    
    # Create a DataFrame from the raw prediction array
    df_tmp = pd.DataFrame(arr, columns=["predicted as 0", "predicted as 1"])

    # Determine the predicted value based on probabilities
    df_tmp["predicted value"] = np.where(df_tmp["predicted as 0"] > df_tmp["predicted as 1"], 0, 1)
    
    # Get the column number of the current medicine in the training labels
    col_num = train_y.columns.get_loc(key)
    
    # Add ground truth values to the DataFrame
    df_tmp["ground truth"] = train_y.iloc[:, col_num].copy().values
    
    TP = ((df_tmp['ground truth'] == 1) & (df_tmp['predicted value'] == 1)).sum()
    FP = ((df_tmp['ground truth'] == 0) & (df_tmp['predicted value'] == 1)).sum()
    FN = ((df_tmp['ground truth'] == 1) & (df_tmp['predicted value'] == 0)).sum()
    TN = ((df_tmp['ground truth'] == 0) & (df_tmp['predicted value'] == 0)).sum()
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    # Calculate TP, FP, FN, TN for the current medicine
    total_tp_train += TP
    total_fp_train += FP
    total_fn_train += FN
    total_tn_train += TN
    
    TrainMedicineDictioanry[key] = {
        "TP" : TP,
        "FP" : FP,
        "FN" : FN,
        "TN" : TN,
        "precision" : precision,
        "recall" : recall,
        "f1-score" : f1score
    }

precision = total_tp_train / (total_tp_train + total_fp_train)
recall = total_tp_train / (total_tp_train + total_fn_train)

TrainMedicineDictioanry["overall"] = {
        "TP" : total_tp_train,
        "FP" : total_fp_train,
        "FN" : total_fn_train,
        "TN" : total_tn_train,
        "precision" : precision,
        "recall" : recall,
        "f1-score" : 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
}

print("TrainMedicineDictioanry:")
for key in TrainMedicineDictioanry:
    print(key, TrainMedicineDictioanry[key])

TrainMedicineDictioanry:
桂枝 {'TP': 262, 'FP': 6, 'FN': 9, 'TN': 360, 'precision': 0.9776119402985075, 'recall': 0.966789667896679, 'f1-score': 0.9721706864564007}
柴胡 {'TP': 202, 'FP': 6, 'FN': 3, 'TN': 426, 'precision': 0.9711538461538461, 'recall': 0.9853658536585366, 'f1-score': 0.9782082324455206}
黃芩 {'TP': 250, 'FP': 2, 'FN': 3, 'TN': 382, 'precision': 0.9920634920634921, 'recall': 0.9881422924901185, 'f1-score': 0.9900990099009901}
茯苓 {'TP': 201, 'FP': 6, 'FN': 7, 'TN': 423, 'precision': 0.9710144927536232, 'recall': 0.9663461538461539, 'f1-score': 0.9686746987951806}
澤瀉 {'TP': 218, 'FP': 2, 'FN': 8, 'TN': 409, 'precision': 0.990909090909091, 'recall': 0.9646017699115044, 'f1-score': 0.9775784753363229}
附子 {'TP': 262, 'FP': 7, 'FN': 6, 'TN': 362, 'precision': 0.9739776951672863, 'recall': 0.9776119402985075, 'f1-score': 0.9757914338919925}
甘草 {'TP': 340, 'FP': 1, 'FN': 21, 'TN': 275, 'precision': 0.9970674486803519, 'recall': 0.9418282548476454, 'f1-score': 0.9686609686609687}
當歸 

### 7.2 Calculate the f1 score of the validation dataset and store the values in ValMedicineDictioanry

In [9]:
# Calculate True Positives (TP), False Positives (FP), True Negatives (TN), and False Negatives (FN) for the training set
total_tp_train = 0
total_fp_train = 0
total_tn_train = 0
total_fn_train = 0

# Create a dictioanry to store all values of a medicine
ValMedicineDictioanry = {}

# Iterate through each medicine's raw prediction array
for key, arr in prediction_val_dict.items():
    
    # Create a DataFrame from the raw prediction array
    df_tmp = pd.DataFrame(arr, columns=["predicted as 0", "predicted as 1"])

    # Determine the predicted value based on probabilities
    df_tmp["predicted value"] = np.where(df_tmp["predicted as 0"] > df_tmp["predicted as 1"], 0, 1)
    
    # Get the column number of the current medicine in the training labels
    col_num = val_y.columns.get_loc(key)
    
    # Add ground truth values to the DataFrame
    df_tmp["ground truth"] = val_y.iloc[:, col_num].copy().values
    
    TP = ((df_tmp['ground truth'] == 1) & (df_tmp['predicted value'] == 1)).sum()
    FP = ((df_tmp['ground truth'] == 0) & (df_tmp['predicted value'] == 1)).sum()
    FN = ((df_tmp['ground truth'] == 1) & (df_tmp['predicted value'] == 0)).sum()
    TN = ((df_tmp['ground truth'] == 0) & (df_tmp['predicted value'] == 0)).sum()
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    # Calculate TP, FP, FN, TN for the current medicine
    total_tp_train += TP
    total_fp_train += FP
    total_fn_train += FN
    total_tn_train += TN
    
    ValMedicineDictioanry[key] = {
        "TP" : TP,
        "FP" : FP,
        "FN" : FN,
        "TN" : TN,
        "precision" : precision,
        "recall" : recall,
        "f1-score" : f1score
    }

precision = total_tp_train / (total_tp_train + total_fp_train)
recall = total_tp_train / (total_tp_train + total_fn_train)

ValMedicineDictioanry["overall"] = {
        "TP" : total_tp_train,
        "FP" : total_fp_train,
        "FN" : total_fn_train,
        "TN" : total_tn_train,
        "precision" : precision,
        "recall" : recall,
        "f1-score" : 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
}

print("ValMedicineDictioanry:")
for key in ValMedicineDictioanry:
    print(key, ValMedicineDictioanry[key])

ValMedicineDictioanry:
桂枝 {'TP': 32, 'FP': 40, 'FN': 34, 'TN': 53, 'precision': 0.4444444444444444, 'recall': 0.48484848484848486, 'f1-score': 0.463768115942029}
柴胡 {'TP': 21, 'FP': 21, 'FN': 35, 'TN': 82, 'precision': 0.5, 'recall': 0.375, 'f1-score': 0.42857142857142855}
黃芩 {'TP': 31, 'FP': 25, 'FN': 36, 'TN': 67, 'precision': 0.5535714285714286, 'recall': 0.4626865671641791, 'f1-score': 0.5040650406504066}
茯苓 {'TP': 23, 'FP': 25, 'FN': 38, 'TN': 73, 'precision': 0.4791666666666667, 'recall': 0.3770491803278688, 'f1-score': 0.4220183486238532}
澤瀉 {'TP': 32, 'FP': 19, 'FN': 40, 'TN': 68, 'precision': 0.6274509803921569, 'recall': 0.4444444444444444, 'f1-score': 0.5203252032520325}
附子 {'TP': 43, 'FP': 29, 'FN': 35, 'TN': 52, 'precision': 0.5972222222222222, 'recall': 0.5512820512820513, 'f1-score': 0.5733333333333334}
甘草 {'TP': 54, 'FP': 41, 'FN': 35, 'TN': 29, 'precision': 0.5684210526315789, 'recall': 0.6067415730337079, 'f1-score': 0.5869565217391305}
當歸 {'TP': 22, 'FP': 40, 'FN': 3

the following were all commented since i have no idead what this is

In [10]:

# file_path_suffix = "each_med_csv"   # type the dir for you to remember where u save the result
# for key, df in result_df_dict.items():
#     print(f"DataFrame for {key}:")
#     #myutil.print_df(df)
#     myutil.df_to_csv(df, save_path=("./result/"+file_path_suffix), file_prefix=key)
   



In [11]:
# # make df for all f1_score of each medicine
# all_f1_df = pd.DataFrame([(key, val['f1_score'], (val['TP']+val['FN'])) for key, val in acc_each_med.items()], columns=['medicine', 'f1_score', 'TP+FN'])
# myutil.df_to_csv(all_f1_df, save_path=("./result/"+file_path_suffix), file_prefix='all_f1_score')

In [12]:
# # save f1_score and TP/FP/TN/TN/FN

# spec_str = "model layer:  32-64-128-64-32 units, activation: relu, optimizer: Adam, learning rate: 0.001, epochs: 1000, batch_size: 32, num_med: all. del_med_under_thres: 0"
# # need to type this spec str each time to record the result

# file_path="./result/1_med_accuracy"
# myutil.dict_to_txt(acc_each_med, save_path=file_path, 
#                    file_prefix="accuracy_each_med",
#                    textbox=spec_str)


# myutil.dict_to_txt(train_set_acc, save_path=file_path, 
#                    file_prefix="accuracy_train_set",
#                    textbox="train set"+spec_str)
